In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm

载入数据

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print('train：',train.shape,'test：',test.shape)

train： (1176, 36) test： (294, 35)


In [3]:
train['Attrition']=train['Attrition'].map({"Yes":1,"No":0})

In [10]:
train.

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeNumber,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,Department,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,JobRole,MaritalStatus,Over18,OverTime,StandardHours


In [4]:
train.head()

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,0,Travel_Rarely,605,Sales,21,3,Life Sciences,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,0,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,0,Travel_Rarely,300,Sales,26,3,Marketing,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,0,Non-Travel,1434,Sales,8,4,Life Sciences,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,1,Travel_Frequently,599,Sales,24,1,Life Sciences,1,...,4,80,0,1,4,3,1,0,1,0


In [5]:
w=train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 36 columns):
user_id                     1176 non-null int64
Age                         1176 non-null int64
Attrition                   1176 non-null int64
BusinessTravel              1176 non-null object
DailyRate                   1176 non-null int64
Department                  1176 non-null object
DistanceFromHome            1176 non-null int64
Education                   1176 non-null int64
EducationField              1176 non-null object
EmployeeCount               1176 non-null int64
EmployeeNumber              1176 non-null int64
EnvironmentSatisfaction     1176 non-null int64
Gender                      1176 non-null object
HourlyRate                  1176 non-null int64
JobInvolvement              1176 non-null int64
JobLevel                    1176 non-null int64
JobRole                     1176 non-null object
JobSatisfaction             1176 non-null int64
MaritalStatus           

编码

In [6]:
train=pd.get_dummies(train, columns = ["BusinessTravel"])
train=pd.get_dummies(train, columns = ["Department"])
train=pd.get_dummies(train, columns = ["EducationField"])
train=pd.get_dummies(train, columns = ["EnvironmentSatisfaction"])
train=pd.get_dummies(train, columns = ["Gender"])
train=pd.get_dummies(train, columns = ["JobRole"])
train=pd.get_dummies(train, columns = ["MaritalStatus"])
train=pd.get_dummies(train, columns = ["OverTime"])
train=pd.get_dummies(train, columns = ["Over18"])

In [7]:
train.describe().T[train.describe().T["std"]==0]

,count,mean,std,min,25%,50%,75%,max
EmployeeCount,1176.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
StandardHours,1176.0,80.0,0.0,80.0,80.0,80.0,80.0,80.0
Over18_Y,1176.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [8]:
train.drop(['Over18_Y','StandardHours','EmployeeCount','user_id'],axis=1,inplace=True)

In [92]:
corrDf=train.corr()

In [93]:
pd.DataFrame(corrDf['Attrition'].sort_values(ascending =False)).head(30)

,Attrition
Attrition,1.000000
OverTime_Yes,0.225425
JobRole_Sales Representative,0.175908
MaritalStatus_Single,0.169450
EnvironmentSatisfaction_1,0.132292
BusinessTravel_Travel_Frequently,0.123936
JobRole_Laboratory Technician,0.105588
Department_Sales,0.080562
EducationField_Technical Degree,0.079830
DistanceFromHome,0.072157


In [160]:
from sklearn.model_selection import train_test_split

In [161]:
X_train, X_test, y_train, y_test=train_test_split(train.drop("Attrition",axis=1),train["Attrition"].values,test_size=0.25,random_state=1)

In [178]:
lr = LogisticRegression(solver="liblinear",C=0.5,max_iter=1000,penalty="l2")
lr.fit(X_train, y_train)
lr.score(X_train,y_train)

0.8877551020408163

In [163]:
from sklearn.feature_selection import SelectFromModel

In [141]:
X_embedded = SelectFromModel(lr,threshold=0.005).fit_transform(X_train,y_train)

In [142]:
X_embedded.shape

(882, 50)

In [151]:
lr.fit(X_embedded, y_train)
lr.score(X_embedded,y_train)

0.8934240362811792

In [117]:
pd.DataFrame(X_embedded)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,38,12,3,2,2,1,2,15,3,3,...,0,0,0,0,0,1,0,0,1,0
1,24,29,1,3,1,1,1,13,3,2,...,0,0,0,0,0,1,0,0,1,0
2,41,1,3,4,3,1,1,16,3,3,...,0,0,0,1,0,0,1,0,1,0
3,26,10,3,3,2,4,1,13,3,1,...,0,0,0,1,0,0,0,1,1,0
4,41,28,4,2,4,2,0,23,4,3,...,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,39,1,3,3,2,4,1,15,3,4,...,0,0,0,0,0,0,1,0,0,1
878,27,8,3,3,3,3,1,15,3,4,...,0,0,0,1,0,0,0,1,1,0
879,32,1,4,3,3,4,1,19,3,3,...,0,0,0,1,0,0,1,0,1,0
880,36,2,4,3,2,2,9,11,3,4,...,1,0,0,0,0,0,0,1,0,1


In [16]:
Attrition=lr.predict_proba(test_scaler)

In [17]:
result = pd.DataFrame(columns=["user_id","Attrition"])

In [18]:
result["Attrition"]=[i[1] for i in Attrition]
result["user_id"]=test.user_id.values

In [19]:
result.to_csv('sampleSubmission_9.csv',index=None)